In [1]:
!pip install requests

In [2]:
!pip install lxml

In [3]:
!pip install selectorlib

In [34]:
!pip install selenium

In [38]:
!pip install useragent

  Created wheel for useragent: filename=useragent-0.1.1-py3-none-any.whl size=142386 sha256=ad6397ffdc3fe99d5eda21ebac952f22478c399905c50b80ffef6e642256d0b6
  Stored in directory: c:\users\andriya tiwary\appdata\local\pip\cache\wheels\e9\7e\ba\fbee016aa103d763e93fd575ef844f71ee4979f20ca37c6367
Successfully built useragent


In [41]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13489 sha256=b1e66f0a115e6dfce023f17703ce21b7e879f4412eac8fd3cd78e842e36a967f
  Stored in directory: c:\users\andriya tiwary\appdata\local\pip\cache\wheels\ed\f7\62\50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [135]:
from selectorlib import Extractor
import requests 
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

In [136]:
search_query="smartphones"

In [137]:
ua = UserAgent()
print(ua.chrome)

Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36


In [138]:
header = {'User-Agent':str(ua.chrome)}
print(header)

{'User-Agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.2117.157 Safari/537.36'}


In [139]:
base_url="https://www.amazon.in/s?k="
url=base_url+search_query

In [140]:
url

'https://www.amazon.in/s?k=smartphones'

In [141]:
search_response=requests.get(url,headers=header)

In [142]:
search_response.status_code

200

In [143]:
# function to get content of the page for required query

cookie={}  # insert request cookies within{}

def getAmazonSearch(search_query,page):
    url=base_url+search_query+'&pageNumber='+str(page)
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

In [144]:
# function to get the contents of individual product pages using ASIN (Amazon Standard Identification Number)

def Searchasin(asin):
    url="https://www.amazon.in/dp/"+asin
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

In [145]:
# function to pass on the link of 'see all reviews' and extract the content

def Searchreviews(review_link):
    url="https://www.amazon.in"+review_link
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

In [146]:
# First page product reviews extraction

In [147]:
product_names=[]
response=getAmazonSearch('smartphones',1)
soup=BeautifulSoup(response.content)

# findall() is used to find all the tags with the mentioned attributes and values in a page

for i in soup.findAll("span",{'class':'a-size-medium a-color-base a-text-normal'}): # tag common for all the names of products
    product_names.append(i.text)  # adding the product names to the list

https://www.amazon.in/s?k=smartphones&pageNumber=1


In [148]:
product_names

['Redmi 9 (Sky Blue, 4GB RAM, 64GB Storage)| 5000 mAh| 2.3GHz Mediatek Helio G35 Octa core Processor',
 'Redmi 9A (Nature Green, 2GB RAM, 32GB Storage) | 2GHz Octa-core Helio G25 Processor | 5000 mAh Battery',
 'Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB Storage)',
 'Redmi Note 9 (Pebble Grey, 4GB RAM 64GB Storage) - 48MP Quad Camera & Full HD+ Display',
 'Redmi 9 (Carbon Black, 4GB RAM, 64GB Storage) | 5000 mAh| 2.3GHz Mediatek Helio G35 Octa core Processor',
 'Oppo A31 (Mystery Black, 6GB RAM, 128GB Storage) with No Cost EMI/Additional Exchange Offers',
 'Samsung Galaxy M31 (Space Black, 6GB RAM, 128GB Storage)',
 'Samsung Galaxy M01 Core (Black, 2GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers',
 'Panasonic Eluga i7 (2GB RAM, 16GB Storage, Finger Print Sensor, 4000mAh Battery) (Black)',
 'Vivo Y91i (Ocean Blue, 2GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers',
 'Vivo Y11 (Mineral Blue, 3GB RAM, 32GB Storage) with No Cost EMI/Additional Exchan

In [149]:
len(product_names)

16

In [150]:
#The method of extracting data-asin numbers are similar to that of product names,
# only the tag details have to be changed in findall()

In [169]:
data_asin=[]
pages=8
for i in range(pages):
    response=getAmazonSearch('smartphones',i)
    soup=BeautifulSoup(response.content)
    for i in soup.findAll("div",{'class':"s-result-item s-asin sg-col sg-col-12-of-12 sg-col-12-of-16 sg-col-16-of-20"}):
         data_asin.append(i['data-asin'])

https://www.amazon.in/s?k=smartphones&pageNumber=0
https://www.amazon.in/s?k=smartphones&pageNumber=1
https://www.amazon.in/s?k=smartphones&pageNumber=2
https://www.amazon.in/s?k=smartphones&pageNumber=3
https://www.amazon.in/s?k=smartphones&pageNumber=4
https://www.amazon.in/s?k=smartphones&pageNumber=5
https://www.amazon.in/s?k=smartphones&pageNumber=6
https://www.amazon.in/s?k=smartphones&pageNumber=7


In [170]:
data_asin[:5]

['B08697N43N', 'B08696XB4B', 'B07HGGYWL6', 'B086977TR6', 'B086985T6R']

In [171]:
len(data_asin)

32

In [172]:
#By passing the data-asin numbers, we can extract the 'see all reviews' link for each product in the page

In [173]:
links=[]
mobile_names=[]
asins=[]
for i in range(len(data_asin)):
        response=Searchasin(data_asin[i])
        soup=BeautifulSoup(response.content)
        mobile=soup.find("span",{"id":"productTitle"})
        mobile_names.append(mobile.string)
        link= soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"})
        links.append(link[0]['href'])
        asins.append(data_asin[i])

https://www.amazon.in/dp/B08697N43N
https://www.amazon.in/dp/B08696XB4B
https://www.amazon.in/dp/B07HGGYWL6
https://www.amazon.in/dp/B086977TR6
https://www.amazon.in/dp/B086985T6R
https://www.amazon.in/dp/B08444S68L
https://www.amazon.in/dp/B089MQ622N
https://www.amazon.in/dp/B07HGN617M
https://www.amazon.in/dp/B07HGJJ559
https://www.amazon.in/dp/B08BV2T63X
https://www.amazon.in/dp/B07KXCKPZZ
https://www.amazon.in/dp/B089MTR9JW
https://www.amazon.in/dp/B07XD2FWBC
https://www.amazon.in/dp/B08T2QR7NK
https://www.amazon.in/dp/B07KX1S161
https://www.amazon.in/dp/B086KCCMCP
https://www.amazon.in/dp/B08697N43N
https://www.amazon.in/dp/B08696XB4B
https://www.amazon.in/dp/B07HGGYWL6
https://www.amazon.in/dp/B086977TR6
https://www.amazon.in/dp/B08444S68L
https://www.amazon.in/dp/B086985T6R
https://www.amazon.in/dp/B089MQ622N
https://www.amazon.in/dp/B07HGJJ559
https://www.amazon.in/dp/B07HGN617M
https://www.amazon.in/dp/B08BV2T63X
https://www.amazon.in/dp/B089MTR9JW
https://www.amazon.in/dp/B07

In [175]:
print(len(links),len(mobile_names),len(asins))
mobile_names

32 32 32


['\n\n\n\n\n\n\n\nRedmi 9 (Sky Blue, 4GB RAM, 64GB Storage)| 5000 mAh| 2.3GHz Mediatek Helio G35 Octa core Processor\n\n\n\n\n\n\n',
 '\n\n\n\n\n\n\n\nRedmi 9A (Nature Green, 2GB RAM, 32GB Storage) | 2GHz Octa-core Helio G25 Processor | 5000 mAh Battery\n\n\n\n\n\n\n',
 '\n\n\n\n\n\n\n\nSamsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB Storage)\n\n\n\n\n\n\n',
 '\n\n\n\n\n\n\n\nRedmi Note 9 (Pebble Grey, 4GB RAM 64GB Storage) - 48MP Quad Camera & Full HD+ Display\n\n\n\n\n\n\n',
 '\n\n\n\n\n\n\n\nRedmi 9 (Carbon Black, 4GB RAM, 64GB Storage) | 5000 mAh| 2.3GHz Mediatek Helio G35 Octa core Processor\n\n\n\n\n\n\n',
 '\n\n\n\n\n\n\n\nOppo A31 (Mystery Black, 6GB RAM, 128GB Storage) with No Cost EMI/Additional Exchange Offers\n\n\n\n\n\n\n',
 '\n\n\n\n\n\n\n\nSamsung Galaxy M01 Core (Black, 2GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers\n\n\n\n\n\n\n',
 '\n\n\n\n\n\n\n\nSamsung Galaxy M31 (Space Black, 6GB RAM, 128GB Storage)\n\n\n\n\n\n\n',
 '\n\n\n\n\n\n\n\nSamsung Gala

In [176]:
# Now we have the 'see all review' links. 
# Using this link along with a page number, we can extract the reviews in any number of pages for all the products

In [177]:
reviews=[]
for j in range(len(asins)):
    for k in range(10):
        response=Searchreviews(links[j]+'&pageNumber='+str(k))
        soup=BeautifulSoup(response.content)
        titles=soup.findAll("a",{'data-hook':"review-title"})
        bodies=soup.findAll("span",{'data-hook':"review-body"})
        stars=soup.findAll("i",{'data-hook':"review-star-rating"})
        for i in range(len(titles)):
            reviews.append([mobile_names[j],asins[j],titles[i].text,  bodies[i].text,  int(stars[i].text[0])])  
print('Number of reviews:',len(reviews))

https://www.amazon.in/Redmi-Sky-Blue-64GB-Storage/product-reviews/B08697N43N/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=0
https://www.amazon.in/Redmi-Sky-Blue-64GB-Storage/product-reviews/B08697N43N/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.in/Redmi-Sky-Blue-64GB-Storage/product-reviews/B08697N43N/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.in/Redmi-Sky-Blue-64GB-Storage/product-reviews/B08697N43N/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.in/Redmi-Sky-Blue-64GB-Storage/product-reviews/B08697N43N/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.in/Redmi-Sky-Blue-64GB-Storage/product-reviews/B08697N43N/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.in/Redmi-Sky-Blue-64GB-Storage/product-reviews/B08697N43N/ref=cm_cr_dp_d_sh

https://www.amazon.in/Oppo-Mystery-Storage-Additional-Exchange/product-reviews/B08444S68L/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.in/Oppo-Mystery-Storage-Additional-Exchange/product-reviews/B08444S68L/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.in/Oppo-Mystery-Storage-Additional-Exchange/product-reviews/B08444S68L/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=6
https://www.amazon.in/Oppo-Mystery-Storage-Additional-Exchange/product-reviews/B08444S68L/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=7
https://www.amazon.in/Oppo-Mystery-Storage-Additional-Exchange/product-reviews/B08444S68L/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=8
https://www.amazon.in/Oppo-Mystery-Storage-Additional-Exchange/product-reviews/B08444S68L/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=9
https://www.amaz

https://www.amazon.in/Vivo-Storage-Additional-Exchange-Offers/product-reviews/B07KXCKPZZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.in/Vivo-Storage-Additional-Exchange-Offers/product-reviews/B07KXCKPZZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=6
https://www.amazon.in/Vivo-Storage-Additional-Exchange-Offers/product-reviews/B07KXCKPZZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=7
https://www.amazon.in/Vivo-Storage-Additional-Exchange-Offers/product-reviews/B07KXCKPZZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=8
https://www.amazon.in/Vivo-Storage-Additional-Exchange-Offers/product-reviews/B07KXCKPZZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=9
https://www.amazon.in/Test-Exclusive_2020_1138-Multi-3GB-Storage/product-reviews/B089MTR9JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=0
https://www.amazon.

https://www.amazon.in/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCCMCP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=6
https://www.amazon.in/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCCMCP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=7
https://www.amazon.in/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCCMCP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=8
https://www.amazon.in/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCCMCP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=9
https://www.amazon.in/Redmi-Sky-Blue-64GB-Storage/product-reviews/B08697N43N/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=0
https://www.amazon.in/Redmi-Sky-Blue-64GB-Storage/product-reviews/B08697N43N/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.in/Redmi-Sky-Bl

https://www.amazon.in/Oppo-Mystery-Storage-Additional-Exchange/product-reviews/B08444S68L/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=9
https://www.amazon.in/Redmi-Carbon-Black-64GB-Storage/product-reviews/B086985T6R/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=0
https://www.amazon.in/Redmi-Carbon-Black-64GB-Storage/product-reviews/B086985T6R/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.in/Redmi-Carbon-Black-64GB-Storage/product-reviews/B086985T6R/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.in/Redmi-Carbon-Black-64GB-Storage/product-reviews/B086985T6R/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.in/Redmi-Carbon-Black-64GB-Storage/product-reviews/B086985T6R/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.in/Redmi-Carbon-Black-64GB-Storage/product

https://www.amazon.in/Test-Exclusive_2020_1138-Multi-3GB-Storage/product-reviews/B089MTR9JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=0
https://www.amazon.in/Test-Exclusive_2020_1138-Multi-3GB-Storage/product-reviews/B089MTR9JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.in/Test-Exclusive_2020_1138-Multi-3GB-Storage/product-reviews/B089MTR9JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.in/Test-Exclusive_2020_1138-Multi-3GB-Storage/product-reviews/B089MTR9JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.in/Test-Exclusive_2020_1138-Multi-3GB-Storage/product-reviews/B089MTR9JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.in/Test-Exclusive_2020_1138-Multi-3GB-Storage/product-reviews/B089MTR9JW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
http

https://www.amazon.in/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCCMCP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.in/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCCMCP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.in/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCCMCP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.in/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCCMCP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.in/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCCMCP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.in/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B086KCCMCP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=6
http

In [178]:
review_data = pd.DataFrame(reviews, columns = ['mobile_names','asin','title', 'body','star'])  
print(review_data.shape)

(3200, 5)


In [179]:
review_data.sample(5)

,mobile_names,asin,title,body,star
165,"\n\n\n\n\n\n\n\nRedmi 9A (Nature Green, 2GB RA...",B08696XB4B,\nGood\n,"\n\n Good, Purchased for my dad.\n\n",3
2621,"\n\n\n\n\n\n\n\nRedmi 9 Power (Mighty Black, 6...",B089MTR9JW,\nWrost and bad product with low performance a...,\n\n Phone having bad quality and hanging iss...,1
850,\n\n\n\n\n\n\n\nSamsung Galaxy M21 (Midnight B...,B07HGJJ559,\nAwful service and worst phone.\n,\n\n Pathetic phone. It is overheating and I ...,1
2203,\n\n\n\n\n\n\n\nSamsung Galaxy M01 Core (Black...,B089MQ622N,\nDon't buy this phone\n,\n\n Its not working. Pls don't buy this phon...,1
2405,\n\n\n\n\n\n\n\nSamsung Galaxy M31 (Space Blac...,B07HGN617M,\nInferior quality and built\n,"\n\n I am using it from 3 days , 6000 mah bat...",1


In [180]:
# Save to CSV

review_data.to_csv('raw_mobile_reviews.csv',index=False)